In [ ]:
import pandas as pd
from pprint import pprint as pp
import ast
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

job_df = pd.read_csv(
    r"C:\Users\student\期末專題\embeddings\104_skills_embedding_1140721.csv"
)
course_df = pd.read_csv(
    r"C:\Users\student\期末專題\embeddings\coursera_skills_embedding_1140721.csv"
)

job_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 86 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   job_id    5 non-null      object 
 1   更新日期      5 non-null      object 
 2   查詢職類      5 non-null      int64  
 3   查詢關鍵字     5 non-null      object 
 4   職務類別      5 non-null      object 
 5   職缺名稱      5 non-null      object 
 6   公司名稱      5 non-null      object 
 7   公司連結      5 non-null      object 
 8   產業別       5 non-null      object 
 9   上班地點      5 non-null      object 
 10  地區        5 non-null      object 
 11  城市        5 non-null      object 
 12  國家        5 non-null      object 
 13  薪資        5 non-null      object 
 14  薪資下限      5 non-null      int64  
 15  薪資上限      5 non-null      int64  
 16  職缺描述      5 non-null      object 
 17  職務需求      0 non-null      float64
 18  工作經歷要求    5 non-null      object 
 19  學歷要求      5 non-null      object 
 20  科系要求      1 non-null      object 
 2

In [ ]:
def recommend_courses_for_job(job_skill_vector_list, course_df, threshold=0.6):

    match_records = []

    for i, course in course_df.iterrows():
        course_skills = course["課程技能向量清單"]
        matched_skills = []

        for job_skill in job_skill_vector_list:
            job_vec = job_skill["向量"]
            job_name = job_skill["技能名"]

            similarities = []
            for course_skill in course_skills:
                sim = cosine_similarity([job_vec], [course_skill["向量"]])[0][0]
                similarities.append(sim)

            if similarities and max(similarities) >= threshold:
                matched_skills.append(
                    {"技能名": job_name, "相似度": round(max(similarities), 3)}
                )

        if matched_skills:
            avg_score = round(np.mean([s["相似度"] for s in matched_skills]), 3)
            match_records.append(
                {
                    "課程名稱": course["課程名稱"],
                    "課程網址": course["課程網址"],
                    "覆蓋技能數": len(matched_skills),
                    "平均相似度": avg_score,
                    "覆蓋技能明細": matched_skills,
                }
            )

    # 所有被覆蓋到的技能
    covered_skills = set()
    for rec in match_records:
        covered_skills.update(skill["技能名"] for skill in rec["覆蓋技能明細"])

    # 所有職缺技能
    job_required_skills = set(skill["技能名"] for skill in job_skill_vector_list)

    # 判斷缺口
    uncovered_skills = list(job_required_skills - covered_skills)
    is_fully_covered = len(uncovered_skills) == 0

    # 根據 覆蓋技能數 → 平均相似度 排序課程
    match_records_sorted = sorted(
        match_records, key=lambda r: (-r["覆蓋技能數"], -r["平均相似度"])
    )

    return {
        "是否全技能被涵蓋": is_fully_covered,
        "職缺技能總數": len(job_required_skills),
        "實際被涵蓋數": len(covered_skills),
        "未被覆蓋技能": uncovered_skills,
        "推薦課程數": len(match_records_sorted),
        "推薦課程清單": match_records_sorted,
    }


def parse_vector_field(raw_field):
    if isinstance(raw_field, str):
        try:
            return ast.literal_eval(raw_field)
        except Exception as e:
            print("⚠️ 向量解析錯誤：", e)
            return []
    elif isinstance(raw_field, list):
        return raw_field
    else:
        return []

In [11]:
job_df["職缺技能向量清單"] = job_df["職缺技能向量清單"].apply(parse_vector_field)
course_df["課程技能向量清單"] = course_df["課程技能向量清單"].apply(parse_vector_field)

In [11]:
job_df.info()
job_df["職缺連結"]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 86 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   job_id    5 non-null      object 
 1   更新日期      5 non-null      object 
 2   查詢職類      5 non-null      int64  
 3   查詢關鍵字     5 non-null      object 
 4   職務類別      5 non-null      object 
 5   職缺名稱      5 non-null      object 
 6   公司名稱      5 non-null      object 
 7   公司連結      5 non-null      object 
 8   產業別       5 non-null      object 
 9   上班地點      5 non-null      object 
 10  地區        5 non-null      object 
 11  城市        5 non-null      object 
 12  國家        5 non-null      object 
 13  薪資        5 non-null      object 
 14  薪資下限      5 non-null      int64  
 15  薪資上限      5 non-null      int64  
 16  職缺描述      5 non-null      object 
 17  職務需求      0 non-null      float64
 18  工作經歷要求    5 non-null      object 
 19  學歷要求      5 non-null      object 
 20  科系要求      1 non-null      object 
 2

0    https://www.104.com.tw/job/8f54d
1    https://www.104.com.tw/job/7p2cn
2    https://www.104.com.tw/job/8cbyv
3    https://www.104.com.tw/job/8b902
4    https://www.104.com.tw/job/8bz2q
Name: 職缺連結, dtype: object

In [12]:
job = job_df.iloc[2]
job_vector_list = job["職缺技能向量清單"]

# job_vector_list
result = recommend_courses_for_job(job_vector_list, course_df)
pp(result)

{'實際被涵蓋數': 11,
 '推薦課程數': 26,
 '推薦課程清單': [{'平均相似度': np.float64(0.649),
             '覆蓋技能數': 6,
             '覆蓋技能明細': [{'技能名': 'Node.js（Node.js）: 熟悉後端服務開發，負責 Backend '
                                '程式設計與擬定開發計畫',
                         '相似度': np.float64(0.702)},
                        {'技能名': ' 管理快取與訊息佇列  \n'
                                'Docker（Docker）: 容器化部署，建立一致的開發與運行環境',
                         '相似度': np.float64(0.609)},
                        {'技能名': ' 進行服務容器化與持續部署  \n'
                                'Distributed Services（分散式服務架構）: 設計與實作微服務架構',
                         '相似度': np.float64(0.614)},
                        {'技能名': ' 進行網路相關問題排查與優化  \n'
                                'DevOps工具與流程（DevOps Tools & Processes）: '
                                '持續整合(CI)、持續部署(CD)、自動化測試與監控',
                         '相似度': np.float64(0.602)},
                        {'技能名': ' 使用雲端服務（AWS, GCP, Digital Ocean）進行服務部署與管理  \n'
                                '系統監控（System Monitoring）: 監控